<a href="https://colab.research.google.com/github/Mini1521/Sentiment-Analysis/blob/main/Using_Microsoft_and_Tesla_Dataset_from_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Data- preprocessing

In [44]:
import pandas as pd

# Load the two CSV files
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Company.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/CompanyValues.csv")

# Merge the data on a common column (e.g., 'day_date')
merged_df = pd.merge(df1, df2, on="ticker_symbol", how="left")  # Use "outer", "left", or "right" if needed

# Save the merged dataset to a new CSV file
merged_df.to_csv("merged_data.csv", index=False)

print("Merged CSV saved as 'merged_data.csv'")

Merged CSV saved as 'merged_data.csv'


In [45]:
import pandas as pd

# Load the dataset (update the filename if needed)
df = pd.read_csv('/content/merged_data.csv')

df.rename(columns={'post_date': 'date'}, inplace=True)

# Convert 'day_date' to datetime format
df['day_date'] = pd.to_datetime(df['day_date'], errors='coerce')

# Filter data for the year 2020
df_2019 = df[df['day_date'].dt.year == 2019]
df_2019 = df_2019.sort_values(by='day_date', ascending=True)

# Separate data for Microsoft (MSFT) and Tesla (TSLA)
msft_data = df_2019[df_2019['ticker_symbol'] == 'MSFT']
tsla_data = df_2019[df_2019['ticker_symbol'] == 'TSLA']

# Drop rows with missing values
msft_data = msft_data.dropna()
tsla_data = tsla_data.dropna()


# print(" Stock Data Date Range:")
# print(f"Start Date: {msft_data['day_date'].min()} | End Date: {msft_data['day_date'].max()}")
# print(f"Start Date: {tsla_data['day_date'].min()} | End Date: {tsla_data['day_date'].max()}")

# Save each dataset separately
msft_data.to_csv("msft_2019.csv", index=False)
tsla_data.to_csv("tsla_2019.csv", index=False)

print("Data preprocessing complete.")
print("Saved: 'msft_2019.csv' and 'tsla_2019.csv'")


Data preprocessing complete.
Saved: 'msft_2019.csv' and 'tsla_2019.csv'


# Social Media - preprocessing

In [46]:
import numpy as np
import pandas as pd

# Load datasets
company_tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Company_Tweet.csv")
tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Tweet.csv")

# Merge tweets with company information
tweets = tweets.merge(company_tweets, how='left', on='tweet_id')

In [47]:
# Convert 'post_date' to datetime format
tweets['date'] = pd.to_datetime(tweets['post_date'], unit='s').dt.date
tweets['date'] = pd.to_datetime(tweets['date'], errors='coerce')
# tweets['time'] = pd.to_datetime(tweets['post_date'], unit='s').dt.time

tweets.to_csv("merged_tweets.csv", index=False)

tweets.drop(columns=['comment_num', 'retweet_num', 'like_num'], inplace=True)

In [48]:
# Filter for Microsoft (MSFT) and Tesla (TSLA)
tweets_filtered = tweets[tweets['ticker_symbol'].isin(['MSFT', 'TSLA'])]

# Keep only tweets from 2019
tweets_filtered = tweets_filtered[tweets_filtered['date'].dt.year == 2019]

# Split into separate datasets
msft_tweets = tweets_filtered[tweets_filtered['ticker_symbol'] == 'MSFT']
tsla_tweets = tweets_filtered[tweets_filtered['ticker_symbol'] == 'TSLA']

# Save to CSV files
msft_tweets.to_csv("msft_tweets_2019.csv", index=False)
tsla_tweets.to_csv("tsla_tweets_2019.csv", index=False)

# Cleaning of microsoft tweets

In [49]:
import re
import pandas as pd
import string
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

# Load Twitter dataset
tweets_df = pd.read_csv("/content/msft_tweets_2019.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [50]:
# rename columns
tweets_df.rename(columns={'body': 'Tweet'}, inplace=True)
tweets_df.rename(columns={'post_date': 'day_date'}, inplace=True)

# Droping Duplicates
tweets_df.drop_duplicates(subset=['Tweet'], inplace=True)

# Handle Missing Values
tweets_df.dropna(subset=['Tweet'], inplace=True)  # Drop rows where 'Tweet' is empty

# Ensure 'date' is in datetime format
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')

tweets_df = tweets_df.sort_values(by='date', ascending=True)

In [51]:
# Initialize preprocessing tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)


In [52]:
# Apply cleaning function
tweets_df['cleaned_tweet'] = tweets_df['Tweet'].astype(str).apply(clean_text)

#Saving the cleaned data for further analysis
tweets_df.to_csv("cleaned_msft_tweets.csv", index=False)

# Cleaning of tesla tweets

In [53]:
# Load Twitter dataset
tweets_df = pd.read_csv("/content/tsla_tweets_2019.csv")

In [54]:
# rename columns
tweets_df.rename(columns={'body': 'Tweet'}, inplace=True)
tweets_df.rename(columns={'post_date': 'day_date'}, inplace=True)

# Droping Duplicates
tweets_df.drop_duplicates(subset=['Tweet'], inplace=True)

# Handle Missing Values
tweets_df.dropna(subset=['Tweet'], inplace=True)  # Drop rows where 'Tweet' is empty

In [55]:
# Ensure 'date' is in datetime format
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')

tweets_df = tweets_df.sort_values(by='date', ascending=True)

In [56]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)

In [57]:
# Apply cleaning function
tweets_df['cleaned_tweet'] = tweets_df['Tweet'].astype(str).apply(clean_text)

#Saving the cleaned data for further analysis
tweets_df.to_csv("cleaned_tsla_tweets.csv", index=False)

# Microsoft Articles - Preprocessing

In [58]:
import re
import pandas as pd
import string
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

# Load the dataset (Update the filename if needed)
article = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/msft_articles.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
# Convert 'date' column to datetime format
article['date'] = pd.to_datetime(article['date'], errors='coerce')

# Filter for only the year 2020
year_2020 = article[article['date'].dt.year == 2020]
year_2020 = year_2020.sort_values(by='date', ascending=True)

# Drop rows with missing values
year_2020 = year_2020.dropna()

In [60]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)

In [61]:
# Apply text cleaning to the 'text' column
year_2020['text'] = year_2020['text'].astype(str).apply(clean_text)

# Save the cleaned dataset
year_2020.to_csv("cleaned_msft_articles.csv", index=False)

# Tesla Articles - Preprocessing

In [62]:
# Load the dataset (Update the filename if needed)
article = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/tsla_articles.csv')

In [63]:
# Convert 'date' column to datetime format
article['date'] = pd.to_datetime(article['date'], errors='coerce')

# Filter for only the year 2020
year_2020 = article[article['date'].dt.year == 2020]
year_2020 = year_2020.sort_values(by='date', ascending=True)

# Drop rows with missing values
year_2020 = year_2020.dropna()

In [64]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)

In [65]:
# Apply text cleaning to the 'text' column
year_2020['text'] = year_2020['text'].astype(str).apply(clean_text)

# Save the cleaned dataset
year_2020.to_csv("cleaned_tsla_articles.csv", index=False)

# Sentiment Analysis- microsoft

In [66]:
!pip install textblob

In [67]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Load datasets
tweets_df = pd.read_csv("/content/cleaned_msft_tweets.csv")
news_df = pd.read_csv("/content/cleaned_msft_articles.csv")

# Ensure text column is string type
tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

In [68]:
# VADER Sentiment Analysis for Tweets
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Function to get sentiment score from VADER
def get_vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply VADER sentiment analysis to tweets
tweets_df['sentiment'] = tweets_df['Tweet'].apply(get_vader_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [69]:
# TextBlob Sentiment Analysis for News

# Function to get sentiment score from TextBlob
def get_textblob_sentiment(text):
    score = TextBlob(text).sentiment.polarity
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply TextBlob sentiment analysis to news articles
news_df['sentiment'] = news_df['text'].apply(get_textblob_sentiment)

# Save sentiment-labeled datasets
tweets_df.to_csv("sentiment_msft_tweets.csv", index=False)
news_df.to_csv("sentiment_msft_news.csv", index=False)

print("Sentiment analysis complete! Results saved.")

Sentiment analysis complete! Results saved.


# Using Traditional ML methods for sentiment Analysis -MSFT

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Load preprocessed datasets
tweets_df = pd.read_csv("/content/sentiment_msft_tweets.csv")
news_df = pd.read_csv("/content/sentiment_msft_news.csv")

# Train-test split for Tweets
X_tweets, X_test_tweets, y_tweets, y_test_tweets = train_test_split(
    tweets_df['cleaned_tweet'], tweets_df['sentiment'], test_size=0.2, random_state=42
)

# Train-test split for News
X_news, X_test_news, y_news, y_test_news = train_test_split(
    news_df['text'], news_df['sentiment'], test_size=0.2, random_state=42
)

# Function to train models
def train_model(X_train, y_train, X_test, y_test, model):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Model {model.__class__.__name__} Accuracy: {accuracy:.4f}")
    return pipeline

# Models to Train
models = {
    'Naïve Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear'),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

# Train models for Tweets
print("\n Training Models for Tweet Sentiment Classification ")
trained_models_tweets = {
    name: train_model(X_tweets, y_tweets, X_test_tweets, y_test_tweets, model)
    for name, model in models.items()
}

# Train models for News
print("\n Training Models for News Sentiment Classification ")
trained_models_news = {
    name: train_model(X_news, y_news, X_test_news, y_test_news, model)
    for name, model in models.items()
}


 Training Models for Tweet Sentiment Classification 
Model MultinomialNB Accuracy: 0.7531
Model SVC Accuracy: 0.8916
Model DecisionTreeClassifier Accuracy: 0.8017
Model RandomForestClassifier Accuracy: 0.8540

 Training Models for News Sentiment Classification 
Model MultinomialNB Accuracy: 0.6032
Model SVC Accuracy: 0.6032
Model DecisionTreeClassifier Accuracy: 0.4603
Model RandomForestClassifier Accuracy: 0.5238


Training Models for Tweet Sentiment Classification
* Model MultinomialNB Accuracy: 0.7532
* Model SVC Accuracy: 0.8911
* Model DecisionTreeClassifier Accuracy: 0.8057
* Model RandomForestClassifier Accuracy: 0.8523

Training Models for News Sentiment Classification
* Model MultinomialNB Accuracy: 0.6032
* Model SVC Accuracy: 0.6032
* Model DecisionTreeClassifier Accuracy: 0.4762
* Model RandomForestClassifier Accuracy: 0.5397



# Using ANN for Sentiment Analysis - MSFT

In [85]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

# Load preprocessed datasets
tweets_df = pd.read_csv("/content/sentiment_msft_tweets.csv")
news_df = pd.read_csv("/content/sentiment_msft_news.csv")

# Train-test split for Tweets
X_tweets, X_test_tweets, y_tweets, y_test_tweets = train_test_split(tweets_df['cleaned_tweet'], tweets_df['sentiment'], test_size=0.2, random_state=42)

# Train-test split for News
X_news, X_test_news, y_news, y_test_news = train_test_split(news_df['text'], news_df['sentiment'], test_size=0.2, random_state=42)

# Convert text into TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000)
X_tweets = vectorizer.fit_transform(X_tweets).toarray()
X_test_tweets = vectorizer.transform(X_test_tweets).toarray()
X_news = vectorizer.fit_transform(X_news).toarray()
X_test_news = vectorizer.transform(X_test_news).toarray()

# Build ANN model
def build_ann(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Convert Sentiment Labels into One-Hot Encoding
y_tweets = to_categorical(y_tweets, num_classes=3)
y_test_tweets = to_categorical(y_test_tweets, num_classes=3)
y_news = to_categorical(y_news, num_classes=3)
y_test_news = to_categorical(y_test_news, num_classes=3)

# Train ANN model
print("\n Training ANN Model for Tweets Sentiment Classification ")
ann_tweet_model = build_ann(X_tweets.shape[1])
ann_tweet_model.fit(X_tweets, y_tweets, epochs=10, batch_size=32, validation_data=(X_test_tweets, y_test_tweets))

# Train ANN for News
print("\n Training ANN Model for News Sentiment Classification ")
ann_news_model = build_ann(X_news.shape[1])
ann_news_model.fit(X_news, y_news, epochs=10, batch_size=32, validation_data=(X_test_news, y_test_news))


y_test_tweets_labels = np.argmax(y_test_tweets, axis=1)
y_pred_tweets = ann_tweet_model.predict(X_test_tweets)
y_pred_tweets = np.argmax(y_pred_tweets, axis=1)  # Convert probabilities to class labels
print(" ANN Tweet Sentiment Classification Report:")   #  ANN Evaluation for Tweets
print(classification_report(y_test_tweets_labels, y_pred_tweets))
print(confusion_matrix(y_test_tweets_labels, y_pred_tweets))

y_test_news_labels = np.argmax(y_test_news, axis=1)
y_pred_news = ann_news_model.predict(X_test_news)
y_pred_news = np.argmax(y_pred_news, axis=1)  # Convert probabilities to class labels
print("\n ANN News Sentiment Classification Report:")  # ANN Evaluation for News
print(classification_report(y_test_news_labels, y_pred_news))
print(confusion_matrix(y_test_news_labels, y_pred_news))



 Training ANN Model for Tweets Sentiment Classification 
Epoch 1/10
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.7269 - loss: 0.6509 - val_accuracy: 0.8769 - val_loss: 0.3592
Epoch 2/10
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.9083 - loss: 0.2782 - val_accuracy: 0.8830 - val_loss: 0.3544
Epoch 3/10
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9339 - loss: 0.2039 - val_accuracy: 0.8882 - val_loss: 0.3550
Epoch 4/10
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9519 - loss: 0.1470 - val_accuracy: 0.8893 - val_loss: 0.3825
Epoch 5/10
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9679 - loss: 0.1012 - val_accuracy: 0.8890 - val_loss: 0.4364
Epoch 6/10
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9799 - loss: 0.0661 - val_accuracy: 0.8871 - val_loss: 0.5024
Epoch 7/10
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.9855 - loss: 0.0451 - val_accuracy: 0.8837 - val_loss: 0.5734
Epoch 8/10
1438/14

# Using LSTM for Sentiment Analysis - MSFT

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SpatialDropout1D, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Load the preprocessed datasets for tweets and news
tweets_df = pd.read_csv('/content/sentiment_msft_tweets.csv')
news_df = pd.read_csv('/content/sentiment_msft_news.csv')

# Replace NaN values with an empty string in both tweet and news data
tweets_df['cleaned_tweet'] = tweets_df['cleaned_tweet'].fillna('')
news_df['text'] = news_df['text'].fillna('')

tweets_df['cleaned_tweet'] = tweets_df['cleaned_tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)



encoder = LabelEncoder()     # Encoding Sentiment Labels (assuming 'sentiment' column with Positive, Negative, Neutral)
tweets_df['encoded_sentiment'] = encoder.fit_transform(tweets_df['sentiment'])
news_df['encoded_sentiment'] = encoder.fit_transform(news_df['sentiment'])

y_tweets = tweets_df['encoded_sentiment']
y_news = news_df['encoded_sentiment']

# Tokenizing the Text Data (using preprocessed text column)
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(tweets_df['cleaned_tweet'])
X_tweets = tokenizer.texts_to_sequences(tweets_df['cleaned_tweet'])
X_tweets = pad_sequences(X_tweets, maxlen=100)

tokenizer.fit_on_texts(news_df['text'])
X_news = tokenizer.texts_to_sequences(news_df['text'])
X_news = pad_sequences(X_news, maxlen=100)

# Step 6: Splitting Data into Training and Testing Sets
X_train_tweets, X_test_tweets, y_train_tweets, y_test_tweets = train_test_split(X_tweets, y_tweets, test_size=0.2, random_state=42)
X_train_news, X_test_news, y_train_news, y_test_news = train_test_split(X_news, y_news, test_size=0.2, random_state=42)

# Building LSTM Model
tweet_model = Sequential()
tweet_model.add(Embedding(input_dim=5000, output_dim=128))
tweet_model.add(SpatialDropout1D(0.2))
tweet_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
tweet_model.add(Dense(3, activation='softmax'))  # 3 classes: Positive, Negative, Neutral

tweet_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tweet_history = tweet_model.fit(X_train_tweets, y_train_tweets, epochs=5, batch_size=64, validation_data=(X_test_tweets, y_test_tweets), verbose=2)

# Building LSTM Model for News Sentiment Classification
news_model = Sequential()
news_model.add(Embedding(input_dim=5000, output_dim=128))
news_model.add(SpatialDropout1D(0.2))
news_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
news_model.add(Dense(3, activation='softmax'))  # 3 classes: Positive, Negative, Neutral

news_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
news_history = news_model.fit(X_train_news, y_train_news, epochs=5, batch_size=64, validation_data=(X_test_news, y_test_news), verbose=2)


y_pred_tweets = tweet_model.predict(X_test_tweets) # Evaluate Tweets Model
y_pred_tweets = np.argmax(y_pred_tweets, axis=1)

y_pred_news = news_model.predict(X_test_news)      # Evaluate News Model
y_pred_news = np.argmax(y_pred_news, axis=1)

print("Tweets Sentiment Classification Report:")
print(classification_report(y_test_tweets, y_pred_tweets))
print(confusion_matrix(y_test_tweets, y_pred_tweets))


print("News Sentiment Classification Report:")
print(classification_report(y_test_news, y_pred_news))
print(confusion_matrix(y_test_news, y_pred_news))

Epoch 1/5
719/719 - 133s - 186ms/step - accuracy: 0.7915 - loss: 0.5508 - val_accuracy: 0.8902 - val_loss: 0.3441
Epoch 2/5
719/719 - 128s - 178ms/step - accuracy: 0.9026 - loss: 0.3082 - val_accuracy: 0.9030 - val_loss: 0.3142
Epoch 3/5
719/719 - 127s - 177ms/step - accuracy: 0.9160 - loss: 0.2696 - val_accuracy: 0.9032 - val_loss: 0.3173
Epoch 4/5
719/719 - 127s - 177ms/step - accuracy: 0.9220 - loss: 0.2476 - val_accuracy: 0.9041 - val_loss: 0.3281
Epoch 5/5
719/719 - 128s - 177ms/step - accuracy: 0.9268 - loss: 0.2290 - val_accuracy: 0.9044 - val_loss: 0.3190
Epoch 1/5
4/4 - 5s - 1s/step - accuracy: 0.4440 - loss: 1.0916 - val_accuracy: 0.5873 - val_loss: 1.0706
Epoch 2/5
4/4 - 1s - 201ms/step - accuracy: 0.4920 - loss: 1.0615 - val_accuracy: 0.5873 - val_loss: 1.0083
Epoch 3/5
4/4 - 1s - 192ms/step - accuracy: 0.4920 - loss: 1.0248 - val_accuracy: 0.5873 - val_loss: 0.9894
Epoch 4/5
4/4 - 1s - 189ms/step - accuracy: 0.4920 - loss: 1.0057 - val_accuracy: 0.5873 - val_loss: 1.0021
E

360/360 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 398ms/step
Tweets Sentiment Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.70      0.75      1522
           1       0.92      0.94      0.93      4818
           2       0.91      0.93      0.92      5160

    accuracy                           0.90     11500
   macro avg       0.88      0.86      0.87     11500
weighted avg       0.90      0.90      0.90     11500

[[1063  202  257]
 [  86 4518  214]
 [ 175  165 4820]]
News Sentiment Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00        17
           2       0.59      1.00      0.74        37

    accuracy                           0.59        63
   macro avg       0.20      0.33      0.25        63
weighted avg       0.34      0.59      0.43        63

[[ 0  0  9]
 [ 0  0 17]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**wait time for compiling is almost 30 mins**

Epoch 1/5

719/719 - 171s - loss: 0.5244 - accuracy: 0.8012 - val_loss: 0.3485 - val_accuracy: 0.8920 - 171s/epoch - 238ms/step

Epoch 2/5

719/719 - 167s - loss: 0.3019 - accuracy: 0.9046 - val_loss: 0.3127 - val_accuracy: 0.9043 - 167s/epoch - 232ms/step

Epoch 3/5

719/719 - 167s - loss: 0.2631 - accuracy: 0.9160 - val_loss: 0.3165 - val_accuracy: 0.9011 - 167s/epoch - 232ms/step

Epoch 4/5

719/719 - 166s - loss: 0.2374 - accuracy: 0.9238 - val_loss: 0.3271 - val_accuracy: 0.9006 - 166s/epoch - 231ms/step

Epoch 5/5

719/719 - 169s - loss: 0.2134 - accuracy: 0.9310 - val_loss: 0.3373 - val_accuracy: 0.9060 - 169s/epoch - 235ms/step

Epoch 1/5

4/4 - 4s - loss: 1.0942 - accuracy: 0.4520 - val_loss: 1.0805 - val_accuracy: 0.5873 - 4s/epoch - 1s/step

Epoch 2/5

4/4 - 1s - loss: 1.0685 - accuracy: 0.4920 - val_loss: 1.0473 - val_accuracy: 0.5873 - 988ms/epoch - 247ms/step

Epoch 3/5

4/4 - 1s - loss: 1.0333 - accuracy: 0.4920 - val_loss: 0.9976 - val_accuracy: 0.5873 - 947ms/epoch - 237ms/step

Epoch 4/5

4/4 - 1s - loss: 1.0035 - accuracy: 0.4920 - val_loss: 0.9927 - val_accuracy: 0.5873 - 946ms/epoch - 236ms/step

Epoch 5/5

4/4 - 1s - loss: 0.9696 - accuracy: 0.4920 - val_loss: 0.9814 - val_accuracy: 0.5873 - 923ms/epoch - 231ms/step

360/360 [==============================] - 9s 24ms/step

2/2 [==============================] - 0s 16ms/step

Tweets Sentiment Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.64      0.74      1522
           1       0.91      0.96      0.93      4818
           2       0.91      0.94      0.92      5160

    accuracy                           0.91     11500

   macro avg       0.89      0.85      0.86     11500

weighted avg       0.90      0.91      0.90     11500


[[ 979  222  321]

 [  62 4605  151]

 [  97  228 4835]]

News Sentiment Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00        17
           2       0.59      1.00      0.74        37

    accuracy                           0.59        63

   macro avg       0.20      0.33      0.25        63

weighted avg       0.34      0.59      0.43        63


[[ 0  0  9]

 [ 0  0 17]

 [ 0  0 37]]

# Sentiment Analysis- tesla

ALGORITHMS WORKS !!!

In [70]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Load datasets
tweets_df = pd.read_csv("/content/cleaned_tsla_tweets.csv")
news_df = pd.read_csv("/content/cleaned_tsla_articles.csv")

# Ensure text column is string type
tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

In [71]:
# VADER Sentiment Analysis for Tweets

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
# Function to get sentiment score from VADER
def get_vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply VADER sentiment analysis to tweets
tweets_df['sentiment'] = tweets_df['Tweet'].apply(get_vader_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [72]:
# TextBlob Sentiment Analysis for News

# Function to get sentiment score from TextBlob
def get_textblob_sentiment(text):
    score = TextBlob(text).sentiment.polarity
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply TextBlob sentiment analysis to news articles
news_df['sentiment'] = news_df['text'].apply(get_textblob_sentiment)
news_df['sentiment'] = news_df['sentiment'].astype(int)

# Save sentiment-labeled datasets
tweets_df.to_csv("sentiment_tsla_tweets.csv", index=False)
news_df.to_csv("sentiment_tsla_news.csv", index=False)

print("Sentiment analysis complete! Results saved.")

Sentiment analysis complete! Results saved.


# Using Traditional ML methods for sentiment analysis

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Load preprocessed datasets
tweets_df = pd.read_csv("/content/sentiment_msft_tweets.csv")
news_df = pd.read_csv("/content/sentiment_msft_news.csv")

tweets_df = tweets_df.sample(min(10000, len(tweets_df)), random_state=42)

# Train-test split for Tweets
X_tweets, X_test_tweets, y_tweets, y_test_tweets = train_test_split(
    tweets_df['cleaned_tweet'], tweets_df['sentiment'], test_size=0.2, random_state=42
)

# Train-test split for News
X_news, X_test_news, y_news, y_test_news = train_test_split(
    news_df['text'], news_df['sentiment'], test_size=0.2, random_state=42
)

# Function to train models
def train_model(X_train, y_train, X_test, y_test, model):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Model {model.__class__.__name__} Accuracy: {accuracy:.4f}")
    return pipeline

# Models to Train
models = {
    'Naïve Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear'),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

# Train models for Tweets
print("\n Training Models for Tweet Sentiment Classification ")
trained_models_tweets = {
    name: train_model(X_tweets, y_tweets, X_test_tweets, y_test_tweets, model)
    for name, model in models.items()
}

# Train models for News
print("\n Training Models for News Sentiment Classification ")
trained_models_news = {
    name: train_model(X_news, y_news, X_test_news, y_test_news, model)
    for name, model in models.items()
}



 Training Models for Tweet Sentiment Classification 
Model MultinomialNB Accuracy: 0.7220
Model SVC Accuracy: 0.8270
Model DecisionTreeClassifier Accuracy: 0.7695
Model RandomForestClassifier Accuracy: 0.7945

 Training Models for News Sentiment Classification 
Model MultinomialNB Accuracy: 0.6032
Model SVC Accuracy: 0.6032
Model DecisionTreeClassifier Accuracy: 0.5079
Model RandomForestClassifier Accuracy: 0.5873


Training Models for Tweet Sentiment Classification
*   Model MultinomialNB Accuracy: 0.7215
*   Model SVC Accuracy: 0.8285
*   Model DecisionTreeClassifier Accuracy: 0.7720
*   Model RandomForestClassifier Accuracy: 0.7880


Training Models for News Sentiment Classification
*   Model MultinomialNB Accuracy: 0.5873
*   Model SVC Accuracy: 0.5556
*   Model DecisionTreeClassifier Accuracy: 0.5397
*   Model RandomForestClassifier Accuracy: 0.5238


# Using ANN for Sentiment Analysis

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

# Load preprocessed datasets
tweets_df = pd.read_csv("/content/sentiment_tsla_tweets.csv")
news_df = pd.read_csv("/content/sentiment_tsla_news.csv")

# Replace NaN values with an empty string in both tweet and news data
tweets_df['cleaned_tweet'] = tweets_df['cleaned_tweet'].fillna('')
news_df['text'] = news_df['text'].fillna('')

tweets_df['cleaned_tweet'] = tweets_df['cleaned_tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

# Reduce dataset size to prevent memory overload
tweets_df = tweets_df.sample(min(50000, len(tweets_df)), random_state=42)

# Train-test split for Tweets
X_tweets, X_test_tweets, y_tweets, y_test_tweets = train_test_split(tweets_df['cleaned_tweet'], tweets_df['sentiment'], test_size=0.2, random_state=42)

# Train-test split for News
X_news, X_test_news, y_news, y_test_news = train_test_split(news_df['text'], news_df['sentiment'], test_size=0.2, random_state=42)

# Convert text into TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000)
X_tweets = vectorizer.fit_transform(X_tweets).toarray()
X_test_tweets = vectorizer.transform(X_test_tweets).toarray()
X_news = vectorizer.fit_transform(X_news).toarray()
X_test_news = vectorizer.transform(X_test_news).toarray()

# Build ANN model
def build_ann(input_dim):
    model = Sequential([
       Input(shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(3, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Convert Sentiment Labels into One-Hot Encoding
y_tweets = to_categorical(y_tweets, num_classes=3)
y_test_tweets = to_categorical(y_test_tweets, num_classes=3)
y_news = to_categorical(y_news, num_classes=3)
y_test_news = to_categorical(y_test_news, num_classes=3)

# Train ANN model
print("\n Training ANN Model for Tweets Sentiment Classification ")
ann_tweet_model = build_ann(X_tweets.shape[1])
ann_tweet_model.fit(X_tweets, y_tweets, epochs=10, batch_size=32, validation_data=(X_test_tweets, y_test_tweets))

# Train ANN for News
print("\n Training ANN Model for News Sentiment Classification ")
ann_news_model = build_ann(X_news.shape[1])
ann_news_model.fit(X_news, y_news, epochs=10, batch_size=32, validation_data=(X_test_news, y_test_news))

y_test_tweets_labels = np.argmax(y_test_tweets, axis=1)
y_pred_tweets = ann_tweet_model.predict(X_test_tweets)
y_pred_tweets = np.argmax(y_pred_tweets, axis=1)  # Convert probabilities to class labels
print(" ANN Tweet Sentiment Classification Report:")   #  ANN Evaluation for Tweets
print(classification_report(y_test_tweets_labels, y_pred_tweets))
print(confusion_matrix(y_test_tweets_labels, y_pred_tweets))

y_test_news_labels = np.argmax(y_test_news, axis=1)
y_pred_news = ann_news_model.predict(X_test_news)
y_pred_news = np.argmax(y_pred_news, axis=1)  # Convert probabilities to class labels
print("\n ANN News Sentiment Classification Report:")  # ANN Evaluation for News
print(classification_report(y_test_news_labels, y_pred_news))
print(confusion_matrix(y_test_news_labels, y_pred_news))



 Training ANN Model for Tweets Sentiment Classification 
Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.5911 - loss: 0.8692 - val_accuracy: 0.8128 - val_loss: 0.5148
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8580 - loss: 0.4208 - val_accuracy: 0.8320 - val_loss: 0.4719
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8844 - loss: 0.3514 - val_accuracy: 0.8327 - val_loss: 0.4725
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8999 - loss: 0.2985 - val_accuracy: 0.8360 - val_loss: 0.4799
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9123 - loss: 0.2628 - val_accuracy: 0.8336 - val_loss: 0.4912
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9258 - loss: 0.2304 - val_accuracy: 0.8295 - val_loss: 0.5232
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9365 - loss: 0.1993 - val_accuracy: 0.8299 - val_loss: 0.5504
Epoch 8/10
1250/1250 ━━━━━━━━━━━

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
 ANN Tweet Sentiment Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      3003
           1       0.85      0.84      0.84      4236
           2       0.76      0.75      0.76      2761

    accuracy                           0.83     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.83      0.83     10000

[[2624  186  193]
 [ 218 3556  462]
 [ 239  449 2073]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

 ANN News Sentiment Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.58      1.00      0.74        21
           2       0.00      0.00      0.00         4

    accuracy                           0.58        36
   macro avg       0.19      0.33      0.25        36
weighted avg       0.34      0.58      0.43        36

[[ 0 11  0]
 [ 0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training ANN Model for Tweets Sentiment Classification
Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.5954 - loss: 0.8592 - val_accuracy: 0.8195 - val_loss: 0.5095
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.8563 - loss: 0.4200 - val_accuracy: 0.8312 - val_loss: 0.4758
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.8834 - loss: 0.3475 - val_accuracy: 0.8335 - val_loss: 0.4737
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.8979 - loss: 0.3051 - val_accuracy: 0.8287 - val_loss: 0.4857
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9065 - loss: 0.2751 - val_accuracy: 0.8326 - val_loss: 0.5032
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9187 - loss: 0.2444 - val_accuracy: 0.8287 - val_loss: 0.5225
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9286 - loss: 0.2177 - val_accuracy: 0.8274 - val_loss: 0.5480
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.9432 - loss: 0.1850 - val_accuracy: 0.8236 - val_loss: 0.5748
Epoch 9/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9535 - loss: 0.1547 - val_accuracy: 0.8241 - val_loss: 0.6109
Epoch 10/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.9621 - loss: 0.1301 - val_accuracy: 0.8237 - val_loss: 0.6525

 Training ANN Model for News Sentiment Classification
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.3411 - loss: 1.0948 - val_accuracy: 0.5556 - val_loss: 1.0709
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6497 - loss: 1.0626 - val_accuracy: 0.6111 - val_loss: 1.0583
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6366 - loss: 1.0318 - val_accuracy: 0.5833 - val_loss: 1.0457
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6529 - loss: 0.9907 - val_accuracy: 0.5833 - val_loss: 1.0327
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6496 - loss: 0.9608 - val_accuracy: 0.5833 - val_loss: 1.0199
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6454 - loss: 0.9265 - val_accuracy: 0.5833 - val_loss: 1.0069
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6603 - loss: 0.8774 - val_accuracy: 0.5833 - val_loss: 0.9939
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6713 - loss: 0.8481 - val_accuracy: 0.5833 - val_loss: 0.9822
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6573 - loss: 0.8101 - val_accuracy: 0.5833 - val_loss: 0.9726
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7021 - loss: 0.7406 - val_accuracy: 0.5833 - val_loss: 0.9644

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

 ANN Tweet Sentiment Classification Report:

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      3003
           1       0.85      0.83      0.84      4236
           2       0.76      0.74      0.75      2761

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000

weighted avg       0.82      0.82      0.82     10000

[[2659  167  177]
 [ 239 3534  463]
 [ 237  480 2044]]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step

 ANN News Sentiment Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.58      1.00      0.74        21
           2       0.00      0.00      0.00         4

    accuracy                           0.58        36
   macro avg       0.19      0.33      0.25        36

weighted avg       0.34      0.58      0.43        36

[[ 0 11  0]
 [ 0 21  0]
 [ 0  4  0]]

# Using LSTM for sentiment analysis

**IT WORKS !!!!!**

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SpatialDropout1D, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Load the preprocessed datasets for tweets and news
tweets_df = pd.read_csv('/content/sentiment_tsla_tweets.csv')  # replace with your actual dataset path
news_df = pd.read_csv('/content/sentiment_tsla_news.csv')  # replace with your actual news dataset path

# Replace NaN values with an empty string in both tweet and news data
tweets_df['cleaned_tweet'] = tweets_df['cleaned_tweet'].fillna('')
news_df['text'] = news_df['text'].fillna('')

tweets_df['cleaned_tweet'] = tweets_df['cleaned_tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)


# Encoding Sentiment Labels (assuming 'sentiment' column with Positive, Negative, Neutral)
encoder = LabelEncoder()
tweets_df['encoded_sentiment'] = encoder.fit_transform(tweets_df['sentiment'])
news_df['encoded_sentiment'] = encoder.fit_transform(news_df['sentiment'])

y_tweets = tweets_df['encoded_sentiment']
y_news = news_df['encoded_sentiment']

#Tokenizing the Text Data (using preprocessed 'cleaned_text')
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(tweets_df['cleaned_tweet'])
X_tweets = tokenizer.texts_to_sequences(tweets_df['cleaned_tweet'])
X_tweets = pad_sequences(X_tweets, maxlen=100)

tokenizer.fit_on_texts(news_df['text'])
X_news = tokenizer.texts_to_sequences(news_df['text'])
X_news = pad_sequences(X_news, maxlen=100)

#Splitting Data into Training and Testing Sets
X_train_tweets, X_test_tweets, y_train_tweets, y_test_tweets = train_test_split(X_tweets, y_tweets, test_size=0.2, random_state=42)
X_train_news, X_test_news, y_train_news, y_test_news = train_test_split(X_news, y_news, test_size=0.2, random_state=42)

#Building ANN Model for Tweets Sentiment Classification
tweet_model = Sequential()
tweet_model.add(Embedding(input_dim=5000, output_dim=128))
tweet_model.add(SpatialDropout1D(0.2))
tweet_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
tweet_model.add(Dense(3, activation='softmax'))  # 3 classes: Positive, Negative, Neutral

tweet_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tweet_history = tweet_model.fit(X_train_tweets, y_train_tweets, epochs=5, batch_size=64, validation_data=(X_test_tweets, y_test_tweets), verbose=2)

#Building ANN Model for News Sentiment Classification
news_model = Sequential()
news_model.add(Embedding(input_dim=5000, output_dim=128))
news_model.add(SpatialDropout1D(0.2))
news_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
news_model.add(Dense(3, activation='softmax'))  # 3 classes: Positive, Negative, Neutral

news_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
news_history = news_model.fit(X_train_news, y_train_news, epochs=5, batch_size=64, validation_data=(X_test_news, y_test_news), verbose=2)

# Step 9: Evaluating the Models
# Evaluate Tweets Model
y_pred_tweets = tweet_model.predict(X_test_tweets)
y_pred_tweets = np.argmax(y_pred_tweets, axis=1)

# Evaluate News Model
y_pred_news = news_model.predict(X_test_news)
y_pred_news = np.argmax(y_pred_news, axis=1)

# Performance Metrics for Tweets
print("Tweets Sentiment Classification Report:")
print(classification_report(y_test_tweets, y_pred_tweets))
print(confusion_matrix(y_test_tweets, y_pred_tweets))

# Performance Metrics for News
print("News Sentiment Classification Report:")
print(classification_report(y_test_news, y_pred_news))
print(confusion_matrix(y_test_news, y_pred_news))

Epoch 1/5
4544/4544 - 1334s - 294ms/step - accuracy: 0.8483 - loss: 0.4363 - val_accuracy: 0.8775 - val_loss: 0.3774
Epoch 2/5
4544/4544 - 1355s - 298ms/step - accuracy: 0.8803 - loss: 0.3644 - val_accuracy: 0.8867 - val_loss: 0.3531
Epoch 3/5
4544/4544 - 1327s - 292ms/step - accuracy: 0.8886 - loss: 0.3426 - val_accuracy: 0.8871 - val_loss: 0.3518
Epoch 4/5
4544/4544 - 1308s - 288ms/step - accuracy: 0.8923 - loss: 0.3281 - val_accuracy: 0.8883 - val_loss: 0.3496
Epoch 5/5
4544/4544 - 1304s - 287ms/step - accuracy: 0.8953 - loss: 0.3164 - val_accuracy: 0.8855 - val_loss: 0.3578
Epoch 1/5
3/3 - 7s - 2s/step - accuracy: 0.4437 - loss: 1.0925 - val_accuracy: 0.5833 - val_loss: 1.0673
Epoch 2/5
3/3 - 1s - 446ms/step - accuracy: 0.5845 - loss: 1.0506 - val_accuracy: 0.5833 - val_loss: 1.0140
Epoch 3/5
3/3 - 1s - 474ms/step - accuracy: 0.5845 - loss: 0.9891 - val_accuracy: 0.5833 - val_loss: 0.9804
Epoch 4/5
3/3 - 2s - 695ms/step - accuracy: 0.5845 - loss: 0.9610 - val_accuracy: 0.5833 - val

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**WAIT TIME FOR COMPILING IS ALMOST 2 HOURS**

Epoch 1/5

4544/4544 - 1295s - 285ms/step - accuracy: 0.8476 - loss: 0.4395 - val_accuracy: 0.8785 - val_loss: 0.3745

Epoch 2/5

4544/4544 - 1354s - 298ms/step - accuracy: 0.8808 - loss: 0.3646 - val_accuracy: 0.8867 - val_loss: 0.3514

Epoch 3/5

4544/4544 - 1314s - 289ms/step - accuracy: 0.8887 - loss: 0.3425 - val_accuracy: 0.8873 - val_loss: 0.3500

Epoch 4/5

4544/4544 - 1305s - 287ms/step - accuracy: 0.8927 - loss: 0.3280 - val_accuracy: 0.8895 - val_loss: 0.3490

Epoch 5/5

4544/4544 - 1299s - 286ms/step - accuracy: 0.8956 - loss: 0.3159 - val_accuracy: 0.8879 - val_loss: 0.3549

Epoch 1/5

3/3 - 8s - 3s/step - accuracy: 0.3803 - loss: 1.0958 - val_accuracy: 0.5833 - val_loss: 1.0691

Epoch 2/5

3/3 - 1s - 377ms/step - accuracy: 0.5845 - loss: 1.0560 - val_accuracy: 0.5833 - val_loss: 1.0241

Epoch 3/5

3/3 - 1s - 254ms/step - accuracy: 0.5845 - loss: 0.9970 - val_accuracy: 0.5833 - val_loss: 0.9730

Epoch 4/5

3/3 - 1s - 395ms/step - accuracy: 0.5845 - loss: 0.9573 - val_accuracy: 0.5833 - val_loss: 0.9622

Epoch 5/5

3/3 - 1s - 434ms/step - accuracy: 0.5845 - loss: 0.9171 - val_accuracy: 0.5833 - val_loss: 0.9695

WARNING:tensorflow:5 out of the last 363 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x79ab5ee69300> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.

2272/2272 ━━━━━━━━━━━━━━━━━━━━ 109s 48ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 470ms/step

Tweets Sentiment Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.81      0.83     20075
           1       0.89      0.94      0.91     21900
           2       0.90      0.90      0.90     30720

    accuracy                           0.89     72695

   macro avg       0.88      0.88      0.88     72695

weighted avg       0.89      0.89      0.89     72695


[[16164  1598  2313]

 [  571 20637   692]

 [ 1979   999 27742]]

News Sentiment Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00        11
           2       0.58      1.00      0.74        21

    accuracy                           0.58        36
   macro avg       0.19      0.33      0.25        36

weighted avg       0.34      0.58      0.43        36

[[ 0  0  4]
 [ 0  0 11]
 [ 0  0 21]]


# Merging - Microsoft


In [73]:
import pandas as pd

# Load datasets
stock_data = pd.read_csv("/content/msft_2019.csv")
tweets_sentiment = pd.read_csv("/content/sentiment_msft_tweets.csv")
news_sentiment = pd.read_csv("/content/sentiment_msft_news.csv")

In [74]:
# Convert date columns to datetime
stock_data['day_date'] = pd.to_datetime(stock_data['day_date'])
tweets_sentiment['date'] = pd.to_datetime(tweets_sentiment['date'])
news_sentiment['date'] = pd.to_datetime(news_sentiment['date'])

In [75]:
# Extract Month-Day format (MM-DD)
stock_data['month_day'] = stock_data['day_date'].dt.strftime('%m-%d')
tweets_sentiment['month_day'] = tweets_sentiment['date'].dt.strftime('%m-%d')
news_sentiment['month_day'] = news_sentiment['date'].dt.strftime('%m-%d')

In [76]:
# # Drop original 'date' columns (Optional)
# stock_data.drop(columns=['day_date'], inplace=True)
# tweets_sentiment.drop(columns=['date'], inplace=True)
# news_sentiment.drop(columns=['date'], inplace=True)

In [77]:
# Stock & Tweets
stock_tweet_merged = stock_data.merge(tweets_sentiment, on='month_day', how='left')
stock_tweet_merged.fillna(0, inplace=True)
stock_tweet_merged = stock_tweet_merged.rename(columns={'sentiment_vader': 'tweet_sentiment'})
stock_tweet_merged.to_csv("Merge_MSFT_TWEET.csv", index=False)

In [78]:
# Stock & News
stock_news_merged = stock_data.merge(news_sentiment, on='month_day', how='left')
stock_news_merged.fillna(0, inplace=True)
stock_news_merged = stock_news_merged.rename(columns={'sentiment': 'news_sentiment'})
stock_news_merged.to_csv("Merge_MSFT_NEWS.csv", index=False)

print(" Merging done! Two separate files created: one for Tweets, one for News.")

 Merging done! Two separate files created: one for Tweets, one for News.


<ipython-input-78-225be7571668>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  stock_news_merged.fillna(0, inplace=True)


# Merging - Tesla

In [79]:
import pandas as pd

# Load datasets (assuming they are already preprocessed)
stock_data = pd.read_csv("/content/tsla_2019.csv")
tweets_sentiment = pd.read_csv("/content/sentiment_tsla_tweets.csv")
news_sentiment = pd.read_csv("/content/sentiment_tsla_news.csv")

In [80]:
# Convert date columns to datetime
stock_data['day_date'] = pd.to_datetime(stock_data['day_date'])
tweets_sentiment['date'] = pd.to_datetime(tweets_sentiment['date'])
news_sentiment['date'] = pd.to_datetime(news_sentiment['date'])

In [81]:
# Extract Month-Day format (MM-DD)
stock_data['month_day'] = stock_data['day_date'].dt.strftime('%m-%d')
tweets_sentiment['month_day'] = tweets_sentiment['date'].dt.strftime('%m-%d')
news_sentiment['month_day'] = news_sentiment['date'].dt.strftime('%m-%d')

In [82]:
# Stock & Tweets
stock_tweet_merged = stock_data.merge(tweets_sentiment, on='month_day', how='left')
stock_tweet_merged.fillna(0, inplace=True)
stock_tweet_merged = stock_tweet_merged.rename(columns={'sentiment_vader': 'tweet_sentiment'})
stock_tweet_merged.to_csv("Merge_TSLA_TWEET.csv", index=False)

In [83]:
# Stock & News
stock_news_merged = stock_data.merge(news_sentiment, on='month_day', how='left')
stock_news_merged.fillna(0, inplace=True)
stock_news_merged = stock_news_merged.rename(columns={'sentiment': 'news_sentiment'})
stock_news_merged.to_csv("Merge_TSLA_NEWS.csv", index=False)

print(" Merging done! Two separate files created: one for Tweets, one for News.")

 Merging done! Two separate files created: one for Tweets, one for News.


<ipython-input-83-df03a4931140>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  stock_news_merged.fillna(0, inplace=True)


# Stock Data Analysis (Microsoft)

In [86]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load and preprocess datasets
stock_tweet = pd.read_csv("/content/Merge_MSFT_TWEET.csv")
stock_news = pd.read_csv("/content/Merge_MSFT_NEWS.csv")

# Convert date column to datetime
stock_tweet['month_day'] = pd.to_datetime(stock_tweet['month_day'], format='%m-%d')
stock_news['month_day'] = pd.to_datetime(stock_news['month_day'], format='%m-%d')

# Sort data chronologically
stock_tweet = stock_tweet.sort_values('month_day')
stock_news = stock_news.sort_values('month_day')

# Feature Engineering: Moving Averages & Volatility
for df in [stock_tweet, stock_news]:
    df['5_day_avg'] = df['close_value'].rolling(window=5).mean()
    df['10_day_avg'] = df['close_value'].rolling(window=10).mean()
    df['volatility'] = df['close_value'].pct_change().rolling(window=5).std()
    # Handle sentiment column name correctly
    if 'sentiment' in df.columns:
        df['sentiment_7d_avg'] = df['sentiment'].rolling(window=7).mean()  # Tweets Sentiment
    elif 'news_sentiment' in df.columns:
        df['sentiment_7d_avg'] = df['news_sentiment'].rolling(window=7).mean()  # News Sentiment
    df.fillna(0, inplace=True)  # Fill missing values

# Save processed data
stock_tweet.to_csv("MSFT_FEATURED_TWEET.csv", index=False)
stock_news.to_csv("MSFT_FEATURED_NEWS.csv", index=False)

print(" Feature Engineering Completed!")

 Feature Engineering Completed!


In [ ]:
# Scaling Features
scaler = MinMaxScaler(feature_range=(0, 1))

# Load Tweet Data
tweet_data = pd.read_csv("MSFT_FEATURED_TWEET.csv")
scaled_data = scaler.fit_transform(tweet_data[['close_value', 'sentiment_7d_avg', '5_day_avg', '10_day_avg', 'volatility']])

# Prepare Data for LSTM (Using 60-day history)
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Build Improved LSTM Model for Tweets
model_tweet = Sequential()
model_tweet.add(LSTM(units=100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model_tweet.add(Dropout(0.3))
model_tweet.add(LSTM(units=100))
model_tweet.add(Dropout(0.3))
model_tweet.add(Dense(units=1))

model_tweet.compile(optimizer='adam', loss='mean_squared_error')
model_tweet.fit(X, y, epochs=20, batch_size=32, verbose=2)

# Save Model
model_tweet.save("LSTM_MSFT_TWEET.h5")
print(" LSTM Model Trained for Tweet Sentiment!")

**wait time for above compiling is almost 1 hour 10 mins or so**

In [ ]:
# Load News Data
news_data = pd.read_csv("MSFT_FEATURED_NEWS.csv")
scaled_news_data = scaler.fit_transform(news_data[['close_value', 'sentiment_7d_avg', '5_day_avg', '10_day_avg', 'volatility']])

# Prepare Data for LSTM
X_news, y_news = [], []
for i in range(60, len(scaled_news_data)):
    X_news.append(scaled_news_data[i-60:i])
    y_news.append(scaled_news_data[i, 0])

X_news, y_news = np.array(X_news), np.array(y_news)

# Build Improved LSTM Model for News
model_news = Sequential()
model_news.add(LSTM(units=100, return_sequences=True, input_shape=(X_news.shape[1], X_news.shape[2])))
model_news.add(Dropout(0.3))
model_news.add(LSTM(units=100))
model_news.add(Dropout(0.3))
model_news.add(Dense(units=1))

model_news.compile(optimizer='adam', loss='mean_squared_error')
model_news.fit(X_news, y_news, epochs=20, batch_size=32, verbose=2)

# Save Model
model_news.save("LSTM_MSFT_NEWS.h5")
print(" LSTM Model Trained for News Sentiment!")

In [ ]:
# Evaluate Models using MAE & RMSE
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predictions
tweet_pred = model_tweet.predict(X)
news_pred = model_news.predict(X_news)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{model_name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

evaluate_model(y, tweet_pred, "LSTM Tweet Model")
evaluate_model(y_news, news_pred, "LSTM News Model")

In [ ]:
# Visualization of Predictions vs Actual
def plot_predictions(actual, predicted, title):
    plt.figure(figsize=(12, 6))
    plt.plot(actual, label="Actual Prices", color='blue')
    plt.plot(predicted, label="Predicted Prices", color='red', linestyle="dashed")
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Stock Price")
    plt.legend()
    plt.show()

plot_predictions(y, tweet_pred, "LSTM Predictions for Tweet Sentiment Impact on Stock Prices")
plot_predictions(y_news, news_pred, "LSTM Predictions for News Sentiment Impact on Stock Prices")

print(" Visualization Complete!")

In [ ]:
import matplotlib.pyplot as plt

# Ensure the date column is in datetime format
stock_tweet['month_day'] = pd.to_datetime(stock_tweet['month_day'])

# Create a figure
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Sentiment Score with its own y-axis
ax1.plot(stock_tweet['month_day'], stock_tweet['sentiment'].rolling(10).mean(),
         label='Sentiment (10-day Avg)', color='blue')
ax1.set_ylabel("Sentiment Score", color='blue')
ax1.set_ylim(-1, 1)  # Limit sentiment scores to a reasonable range

# Create a second y-axis for Stock Price
ax2 = ax1.twinx()
ax2.plot(stock_tweet['month_day'], stock_tweet['close_value'],
         label='Stock Price', color='orange', alpha=0.7)
ax2.set_ylabel("Stock Price", color='orange')

# Improve formatting
ax1.set_xlabel("Date")
plt.title("Microsoft Stock Price vs Sentiment Trend")

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()


# Stock Data Analysis (Tesla)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load and preprocess datasets
stock_tweet = pd.read_csv("/content/Merge_TSLA_TWEET.csv")
stock_news = pd.read_csv("/content/Merge_TSLA_NEWS.csv")

# Convert date column to datetime
stock_tweet['month_day'] = pd.to_datetime(stock_tweet['month_day'], format='%m-%d')
stock_news['month_day'] = pd.to_datetime(stock_news['month_day'], format='%m-%d')

# Sort data chronologically
stock_tweet = stock_tweet.sort_values('month_day')
stock_news = stock_news.sort_values('month_day')

# Feature Engineering: Moving Averages & Volatility
for df in [stock_tweet, stock_news]:
    df['5_day_avg'] = df['close_value'].rolling(window=5).mean()
    df['10_day_avg'] = df['close_value'].rolling(window=10).mean()
    df['volatility'] = df['close_value'].pct_change().rolling(window=5).std()
    # Handle sentiment column name correctly
    if 'sentiment' in df.columns:
        df['sentiment_7d_avg'] = df['sentiment'].rolling(window=7).mean()  # Tweets Sentiment
    elif 'news_sentiment' in df.columns:
        df['sentiment_7d_avg'] = df['news_sentiment'].rolling(window=7).mean()  # News Sentiment
    df.fillna(0, inplace=True)  # Fill missing values

# Save processed data
stock_tweet.to_csv("TSLA_FEATURED_TWEET.csv", index=False)
stock_news.to_csv("TSLA_FEATURED_NEWS.csv", index=False)

print(" Feature Engineering Completed!")

In [ ]:
# Load Tweet Data
tweet_data = pd.read_csv("/content/TSLA_FEATURED_TWEET.csv")

# Scale Features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(tweet_data[['close_value', 'sentiment_7d_avg', '5_day_avg', '10_day_avg', 'volatility']])

# Prepare Data for LSTM
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Build LSTM Model
model_tweet = Sequential()
model_tweet.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model_tweet.add(Dropout(0.2))
model_tweet.add(LSTM(units=50))
model_tweet.add(Dropout(0.2))
model_tweet.add(Dense(units=1))

model_tweet.compile(optimizer='adam', loss='mean_squared_error')
model_tweet.fit(X, y, epochs=7, batch_size=32)

# Save Model
model_tweet.save("LSTM_TSLA_TWEET.h5")

print(" LSTM Model Trained for Tweet Sentiment!")

**wait time for above compiling is almost 1hr 40 mins**

In [ ]:
# Load News Data
news_data = pd.read_csv("/content/TSLA_FEATURED_NEWS.csv")

# Scale Features
scaled_news_data = scaler.fit_transform(news_data[['close_value', 'sentiment_7d_avg', '5_day_avg', '10_day_avg', 'volatility']])

# Prepare Data for LSTM
X_news, y_news = [], []
for i in range(60, len(scaled_news_data)):
    X_news.append(scaled_news_data[i-60:i])
    y_news.append(scaled_news_data[i, 0])

X_news, y_news = np.array(X_news), np.array(y_news)

# Build LSTM Model
model_news = Sequential()
model_news.add(LSTM(units=50, return_sequences=True, input_shape=(X_news.shape[1], X_news.shape[2])))
model_news.add(Dropout(0.2))
model_news.add(LSTM(units=50))
model_news.add(Dropout(0.2))
model_news.add(Dense(units=1))

model_news.compile(optimizer='adam', loss='mean_squared_error')
model_news.fit(X_news, y_news, epochs=25, batch_size=32)

# Save Model
model_news.save("LSTM_TSLA_NEWS.h5")

print("LSTM Model Trained for News Sentiment!")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predictions
tweet_pred = model_tweet.predict(X)
news_pred = model_news.predict(X_news)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{model_name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

evaluate_model(y, tweet_pred, "LSTM Tweet Model")
evaluate_model(y_news, news_pred, "LSTM News Model")

In [ ]:
# Visualization of Predictions vs Actual
def plot_predictions(actual, predicted, title):
    plt.figure(figsize=(12, 6))
    plt.plot(actual, label="Actual Prices", color='blue')
    plt.plot(predicted, label="Predicted Prices", color='red', linestyle="dashed")
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Stock Price")
    plt.legend()
    plt.show()

plot_predictions(y, tweet_pred, "LSTM Predictions for Tweet Sentiment Impact on Stock Prices")
plot_predictions(y_news, news_pred, "LSTM Predictions for News Sentiment Impact on Stock Prices")

print("Visualization Complete!")